In [ ]:
#- Prerequisites - all files in their album folders are located inside images/ folder

In [ ]:
import os
import re
from collections import defaultdict

# === Папка с альбомами ===
ROOT_DIR = "images"
album_dirs = [d for d in os.listdir(ROOT_DIR) if os.path.isdir(os.path.join(ROOT_DIR, d))]

# === Инициализация счётчиков ===
stats = {}
total = defaultdict(int)

# === Шаблоны ===
pattern_copy = re.compile(r"\s\(\d+\)| copy", re.IGNORECASE)

# === Анализ по альбомам ===
for album in album_dirs:
    folder_path = os.path.join(ROOT_DIR, album)
    files = os.listdir(folder_path)

    counts = defaultdict(int)

    for file in files:
        file_path = os.path.join(folder_path, file)
        if not os.path.isfile(file_path):
            continue

        ext = os.path.splitext(file)[1].lower()

        if ext in ['.jpg', '.jpeg', '.heic']:
            if '-edited' in file.lower():
                counts['edited'] += 1
            else:
                counts[ext] += 1
                if pattern_copy.search(file):
                    counts['renamed'] += 1
        elif ext == '.json':
            counts['json'] += 1
        else:
            counts['other'] += 1

    stats[album] = dict(counts)
    for k, v in counts.items():
        total[k] += v

# === Вывод статистики ===
print("\n📊 Статистика по альбомам:")
for album, counts in stats.items():
    print(f"\n📁 {album}:")
    for k, v in counts.items():
        print(f"  {k}: {v}")

# === Подсчёт общего количества пригодных фото ===
photo_exts = ['.jpg', '.jpeg', '.heic']
total_photos = sum(total[ext] for ext in photo_exts)
total_edited = total['edited']
usable_photos = total_photos  # editable уже исключены выше, не учитываются

print("\n📈 Итого по всем альбомам:")
for k, v in total.items():
    print(f"  {k}: {v}")

print(f"\n🧮 Всего фото-файлов (без -edited): {total_photos}")
print(f"✂️  Файлов с '-edited' в имени:        {total_edited}")
print(f"✅ Годных для обработки файлов:        {usable_photos}")

In [ ]:
import os
import re
import math
import json
import pandas as pd
from datetime import datetime
from PIL import Image, ExifTags
from tqdm import tqdm

# === Paths ===
ROOT = os.path.abspath(".")
SRC_FOLDER = os.path.join(ROOT, "images/PhotoMap 2022-2025")
OUT_FOLDER = os.path.join(ROOT, "photos")
CSV_FOLDER = os.path.join(ROOT, "csv")
CSV_MAIN = os.path.join(CSV_FOLDER, "photos.csv")
CSV_MISSING = os.path.join(CSV_FOLDER, "missing_coords.csv")

os.makedirs(OUT_FOLDER, exist_ok=True)
os.makedirs(CSV_FOLDER, exist_ok=True)

# === Helpers ===
def convert_to_degrees(v):
    d, m, s = v
    return float(d) + float(m) / 60 + float(s) / 3600

def extract_gps_from_exif(image_path):
    try:
        image = Image.open(image_path)
        exif_data = image._getexif()
        if not exif_data:
            return None, None

        gps_info = {}
        date_str = None
        for tag, value in exif_data.items():
            decoded = ExifTags.TAGS.get(tag)
            if decoded == "GPSInfo":
                for t in value:
                    sub_decoded = ExifTags.GPSTAGS.get(t)
                    gps_info[sub_decoded] = value[t]
            elif decoded == "DateTimeOriginal":
                date_str = value

        if not gps_info:
            return None, date_str

        lat = convert_to_degrees(gps_info.get("GPSLatitude"))
        if gps_info.get("GPSLatitudeRef") == "S":
            lat = -lat
        lon = convert_to_degrees(gps_info.get("GPSLongitude"))
        if gps_info.get("GPSLongitudeRef") == "W":
            lon = -lon

        return (lat, lon), date_str
    except:
        return None, None

def extract_gps_from_json(json_path):
    try:
        with open(json_path, "r", encoding="utf-8") as f:
            data = json.load(f)
            geo = data.get("geoData")
            photo_taken_time = data.get("photoTakenTime", {}).get("timestamp")
            if geo:
                lat = geo.get("latitude")
                lon = geo.get("longitude")
                return (lat, lon), photo_taken_time
    except:
        pass
    return None, None

def find_json_path_for_image(image_path):
    base = os.path.basename(image_path)
    folder = os.path.dirname(image_path)

    suffixes = [
        ".supplemental-metadata.json",
        ".supplemental-meta.json",
        ".supplemental-metada.json",
        ".supplemental-metadat.json",
        ".supplemental-me.json"
    ]

    for suffix in suffixes:
        candidate = image_path + suffix
        if os.path.exists(candidate):
            return candidate

    match = re.match(r'^(.*)\((\d+)\)\.(jpg|jpeg|heic)$', base, re.IGNORECASE)
    if match:
        base_clean = match.group(1).strip()
        suffix_num = match.group(2)
        ext = match.group(3)
        for sfx in suffixes:
            sfx_with_index = sfx.replace(".json", f"({suffix_num}).json")
            alt_json = os.path.join(folder, f"{base_clean}.{ext}{sfx_with_index}")
            if os.path.exists(alt_json):
                return alt_json

    return None

def parse_date_components(date_str):
    try:
        if date_str and len(str(date_str)) == 10 and str(date_str).isdigit():
            dt = datetime.fromtimestamp(int(date_str))
        else:
            dt = datetime.strptime(date_str, "%Y:%m:%d %H:%M:%S")
        return str(dt.year), f"{dt.month:02d}", f"{dt.day:02d}"
    except:
        return "2099", "01", "01"

def spiral_coords(lat, lon, index, step=0.0002):
    angle = index * (math.pi / 3)
    radius = step * (1 + index // 6)
    return lat + radius * math.cos(angle), lon + radius * math.sin(angle)

# === Find all images ===
image_files = []
for dirpath, _, filenames in os.walk(SRC_FOLDER):
    for f in filenames:
        if f.lower().endswith(('.jpg', '.jpeg', '.heic')) and "-edited" not in f.lower():
            image_files.append(os.path.join(dirpath, f))

# === Processing ===
Image.init()
coords_seen = {}
records_ok = []
records_missing = []

for in_path in tqdm(image_files, desc="\U0001f4e6 Обработка изображений", ncols=80):
    source_filename = os.path.basename(in_path)
    ext = os.path.splitext(source_filename)[1].lower()
    album = os.path.basename(os.path.dirname(in_path))

    try:
        image = Image.new("RGB", (1, 1), (255, 255, 255))

        lat, lon, date_str = None, None, None
        source_type = None

        if ext in [".jpg", ".jpeg"]:
            gps, date_str = extract_gps_from_exif(in_path)
            if gps:
                lat, lon = gps
                source_type = "exif"

        if lat is None or lon is None:
            json_path = find_json_path_for_image(in_path)
            if json_path:
                gps_json, json_date = extract_gps_from_json(json_path)
                if gps_json is not None:
                    try:
                        lat = float(gps_json[0])
                        lon = float(gps_json[1])
                        source_type = "json"
                    except:
                        lat, lon = None, None
                if not date_str and json_date:
                    date_str = json_date

        year, month, day = parse_date_components(date_str)
        base_name = f"IMG_{year}{month}{day}_{abs(hash(source_filename)) % 10**6}.jpg"
        out_path = os.path.join(OUT_FOLDER, base_name)
        image.save(out_path, "JPEG", quality=85)

        row = {
            'filename': base_name,
            'folder': os.path.basename(OUT_FOLDER),
            'latitude': lat,
            'longitude': lon,
            'year': year,
            'month': month,
            'day': day,
            'album': album,
            'source_filename': source_filename,
            'source_type': source_type
        }

        if lat in (None, 0.0) or lon in (None, 0.0):
            row['latitude'] = None
            row['longitude'] = None
            records_missing.append(row)
        else:
            key = (round(lat, 6), round(lon, 6))
            count = coords_seen.get(key, 0)
            if count > 0:
                row['latitude'], row['longitude'] = spiral_coords(lat, lon, count)
            coords_seen[key] = count + 1
            records_ok.append(row)

    except Exception as e:
        year, month, day = "2099", "01", "01"
        base_name = f"IMG_{year}{month}{day}_{abs(hash(source_filename)) % 10**6}.jpg"
        out_path = os.path.join(OUT_FOLDER, base_name)
        records_missing.append({
            'filename': base_name,
            'folder': os.path.basename(OUT_FOLDER),
            'latitude': None,
            'longitude': None,
            'year': year,
            'month': month,
            'day': day,
            'album': album,
            'source_filename': source_filename,
            'source_type': None
        })

# === Save CSVs ===
pd.DataFrame(records_ok).to_csv(CSV_MAIN, index=False)
pd.DataFrame(records_missing).to_csv(CSV_MISSING, index=False)

# === Summary ===
print(f"\n\U0001f4ca Сводка:")
print(f"\U0001f9ee Всего обработано: {len(records_ok) + len(records_missing)}")
print(f"✅ С координатами:   {len(records_ok)}")
print(f"❌ Без координат:    {len(records_missing)}")
print(f"📄 CSV-файлы:        {CSV_MAIN}, {CSV_MISSING}")
print(f"📁 Изображения:      {OUT_FOLDER}")


In [ ]:
# --BACKUP

In [18]:
import os
import re
import math
import json
import pandas as pd
import subprocess
from datetime import datetime
from PIL import Image, ExifTags
from tqdm import tqdm

# === Paths ===
ROOT = os.path.abspath(".")
SRC_FOLDER = os.path.join(ROOT, "images/PhotoMap 2022-2025")
OUT_FOLDER = os.path.join(ROOT, "photos")
CSV_FOLDER = os.path.join(ROOT, "csv")
CSV_MAIN = os.path.join(CSV_FOLDER, "photos.csv")
CSV_MISSING = os.path.join(CSV_FOLDER, "missing_coords.csv")

os.makedirs(OUT_FOLDER, exist_ok=True)
os.makedirs(CSV_FOLDER, exist_ok=True)

# === Helpers ===
def convert_to_degrees(v):
    d, m, s = v
    return float(d) + float(m) / 60 + float(s) / 3600

def extract_gps_from_exif(image_path):
    try:
        image = Image.open(image_path)
        exif_data = image._getexif()
        if not exif_data:
            return None, None

        gps_info = {}
        date_str = None
        for tag, value in exif_data.items():
            decoded = ExifTags.TAGS.get(tag)
            if decoded == "GPSInfo":
                for t in value:
                    sub_decoded = ExifTags.GPSTAGS.get(t)
                    gps_info[sub_decoded] = value[t]
            elif decoded == "DateTimeOriginal":
                date_str = value

        if not gps_info:
            return None, date_str

        lat = convert_to_degrees(gps_info.get("GPSLatitude"))
        if gps_info.get("GPSLatitudeRef") == "S":
            lat = -lat
        lon = convert_to_degrees(gps_info.get("GPSLongitude"))
        if gps_info.get("GPSLongitudeRef") == "W":
            lon = -lon

        return (lat, lon), date_str
    except:
        return None, None

def extract_gps_from_json(json_path):
    try:
        with open(json_path, "r", encoding="utf-8") as f:
            data = json.load(f)
            geo = data.get("geoData")
            photo_taken_time = data.get("photoTakenTime", {}).get("timestamp")
            if geo:
                lat = geo.get("latitude")
                lon = geo.get("longitude")
                return (lat, lon), photo_taken_time
    except:
        pass
    return None, None

def find_json_path_for_image(image_path):
    base = os.path.basename(image_path)
    folder = os.path.dirname(image_path)

    suffixes = [
        ".supplemental-metadata.json",
        ".supplemental-meta.json",
        ".supplemental-metada.json",
        ".supplemental-metadat.json",
        ".supplemental-me.json"
    ]

    for suffix in suffixes:
        candidate = image_path + suffix
        if os.path.exists(candidate):
            return candidate

    match = re.match(r'^(.*)\((\d+)\)\.(jpg|jpeg|heic)$', base, re.IGNORECASE)
    if match:
        base_clean = match.group(1).strip()
        suffix_num = match.group(2)
        ext = match.group(3)
        for sfx in suffixes:
            sfx_with_index = sfx.replace(".json", f"({suffix_num}).json")
            alt_json = os.path.join(folder, f"{base_clean}.{ext}{sfx_with_index}")
            if os.path.exists(alt_json):
                return alt_json

    return None

def parse_date_components(date_str):
    try:
        if date_str and len(str(date_str)) == 10 and str(date_str).isdigit():
            dt = datetime.fromtimestamp(int(date_str))
        else:
            dt = datetime.strptime(date_str, "%Y:%m:%d %H:%M:%S")
        return str(dt.year), f"{dt.month:02d}", f"{dt.day:02d}"
    except:
        return "2099", "01", "01"

def spiral_coords(lat, lon, index, step=0.0002):
    angle = index * (math.pi / 3)
    radius = step * (1 + index // 6)
    return lat + radius * math.cos(angle), lon + radius * math.sin(angle)

# === Find all images ===
image_files = []
for dirpath, _, filenames in os.walk(SRC_FOLDER):
    for f in filenames:
        if f.lower().endswith(('.jpg', '.jpeg', '.heic')) and "-edited" not in f.lower():
            image_files.append(os.path.join(dirpath, f))

# === Processing ===
Image.init()
coords_seen = {}
records_ok = []
records_missing = []

for in_path in tqdm(image_files, desc="\U0001f4e6 Обработка изображений", ncols=80):
    source_filename = os.path.basename(in_path)
    ext = os.path.splitext(source_filename)[1].lower()
    album = os.path.basename(os.path.dirname(in_path))
    source_type = None

    try:
        lat, lon, date_str = None, None, None

        if ext in [".jpg", ".jpeg"]:
            gps, date_str = extract_gps_from_exif(in_path)
            if gps:
                lat, lon = gps
                source_type = "exif"

        if lat is None or lon is None:
            json_path = find_json_path_for_image(in_path)
            if json_path:
                gps_json, json_date = extract_gps_from_json(json_path)
                if gps_json is not None:
                    try:
                        lat = float(gps_json[0])
                        lon = float(gps_json[1])
                        source_type = (source_type or "") + "+json"
                        if ext == ".heic":
                            print(f"[HEIC] Найден JSON → lat: {lat}, lon: {lon}, file: {source_filename}")
                    except:
                        lat, lon = None, None
                if not date_str and json_date:
                    date_str = json_date

        year, month, day = parse_date_components(date_str)
        base_name = f"IMG_{year}{month}{day}_{abs(hash(source_filename)) % 10**6}.jpg"
        out_path = os.path.join(OUT_FOLDER, base_name)

        if ext == ".heic":
            print(f"[HEIC] Конвертация: {source_filename} → {base_name}")
            result = subprocess.run([
                "sips", "-s", "format", "jpeg", in_path, "--out", out_path
            ], capture_output=True, text=True)
            if result.returncode != 0:
                print(f"[HEIC] ❌ Ошибка sips: {result.stderr.strip()}")
                image = Image.new("RGB", (800, 600), (128, 128, 128))
                image.save(out_path, "JPEG", quality=85)
                source_type = (source_type or "") + "+heic-error"
            else:
                image = Image.open(out_path)
        else:
            image = Image.open(in_path)
            width, height = image.size
            new_size = (int(width * 0.8), int(height * 0.8))
            image = image.resize(new_size, Image.LANCZOS)
            image.convert("RGB").save(out_path, "JPEG", quality=85)

        row = {
            'filename': base_name,
            'folder': os.path.basename(OUT_FOLDER),
            'latitude': lat,
            'longitude': lon,
            'year': year,
            'month': month,
            'day': day,
            'album': album,
            'source_filename': source_filename,
            'source_type': source_type
        }

        if lat in (None, 0.0) or lon in (None, 0.0):
            row['latitude'] = None
            row['longitude'] = None
            records_missing.append(row)
        else:
            key = (round(lat, 6), round(lon, 6))
            count = coords_seen.get(key, 0)
            if count > 0:
                row['latitude'], row['longitude'] = spiral_coords(lat, lon, count)
            coords_seen[key] = count + 1
            records_ok.append(row)

    except Exception as e:
        year, month, day = "2099", "01", "01"
        base_name = f"IMG_{year}{month}{day}_{abs(hash(source_filename)) % 10**6}.jpg"
        records_missing.append({
            'filename': base_name,
            'folder': os.path.basename(OUT_FOLDER),
            'latitude': None,
            'longitude': None,
            'year': year,
            'month': month,
            'day': day,
            'album': album,
            'source_filename': source_filename,
            'source_type': None
        })

# === Save CSVs ===
pd.DataFrame(records_ok).to_csv(CSV_MAIN, index=False)
pd.DataFrame(records_missing).to_csv(CSV_MISSING, index=False)

# === Summary ===
print(f"\n\U0001f4ca Сводка:")
print(f"\U0001f9ee Всего обработано: {len(records_ok) + len(records_missing)}")
print(f"✅ С координатами:   {len(records_ok)}")
print(f"❌ Без координат:    {len(records_missing)}")
print(f"📄 CSV-файлы:        {CSV_MAIN}, {CSV_MISSING}")
print(f"📁 Изображения:      {OUT_FOLDER}")


📦 Обработка изображений:   1%|▏                | 7/653 [00:02<03:41,  2.91it/s]

[HEIC] Найден JSON → lat: 50.0842092, lon: 14.4240219, file: IMG_1894.HEIC
[HEIC] Конвертация: IMG_1894.HEIC → IMG_20220501_332311.jpg


📦 Обработка изображений:  19%|██▉            | 127/653 [00:36<01:51,  4.70it/s]

[HEIC] Найден JSON → lat: 50.082902600000004, lon: 14.422433299999998, file: IMG_1899.HEIC
[HEIC] Конвертация: IMG_1899.HEIC → IMG_20220501_71799.jpg


📦 Обработка изображений:  42%|██████▎        | 275/653 [01:21<01:32,  4.08it/s]

...

[HEIC] Найден JSON → lat: 50.075538099999996, lon: 14.437800500000002, file: IMG_1890.HEIC
[HEIC] Конвертация: IMG_1890.HEIC → IMG_20220501_429680.jpg


📦 Обработка изображений:  65%|█████████▊     | 426/653 [02:05<01:09,  3.27it/s]

[HEIC] Найден JSON → lat: 50.0861017, lon: 14.416173599999997, file: IMG_1901.HEIC
[HEIC] Конвертация: IMG_1901.HEIC → IMG_20220501_435404.jpg


📦 Обработка изображений: 100%|███████████████| 653/653 [03:14<00:00,  3.35it/s]


📊 Сводка:
🧮 Всего обработано: 653
✅ С координатами:   653
❌ Без координат:    0
📄 CSV-файлы:        /Users/mloktionov/PycharmProjects/PhotoMaps/csv/photos.csv, /Users/mloktionov/PycharmProjects/PhotoMaps/csv/missing_coords.csv
📁 Изображения:      /Users/mloktionov/PycharmProjects/PhotoMaps/photos


In [ ]:
# -- MAIN script

In [28]:
import os
import re
import math
import json
import pandas as pd
import subprocess
from datetime import datetime
from PIL import Image, ExifTags
from tqdm import tqdm

# === Paths ===
ROOT = os.path.abspath(".")
SRC_FOLDER = os.path.join(ROOT, "images")
OUT_FOLDER = os.path.join(ROOT, "photos")
CSV_FOLDER = os.path.join(ROOT, "csv")
CSV_MAIN = os.path.join(CSV_FOLDER, "photos.csv")
CSV_MISSING = os.path.join(CSV_FOLDER, "missing_coords.csv")

os.makedirs(OUT_FOLDER, exist_ok=True)
os.makedirs(CSV_FOLDER, exist_ok=True)

# === Helpers ===
def convert_to_degrees(v):
    d, m, s = v
    return float(d) + float(m) / 60 + float(s) / 3600

def extract_gps_from_exif(image_path):
    try:
        image = Image.open(image_path)
        exif_data = image._getexif()
        if not exif_data:
            return None, None

        gps_info = {}
        date_str = None
        for tag, value in exif_data.items():
            decoded = ExifTags.TAGS.get(tag)
            if decoded == "GPSInfo":
                for t in value:
                    sub_decoded = ExifTags.GPSTAGS.get(t)
                    gps_info[sub_decoded] = value[t]
            elif decoded == "DateTimeOriginal":
                date_str = value

        if not gps_info:
            return None, date_str

        lat = convert_to_degrees(gps_info.get("GPSLatitude"))
        if gps_info.get("GPSLatitudeRef") == "S":
            lat = -lat
        lon = convert_to_degrees(gps_info.get("GPSLongitude"))
        if gps_info.get("GPSLongitudeRef") == "W":
            lon = -lon

        return (lat, lon), date_str
    except:
        return None, None

def extract_gps_from_json(json_path):
    try:
        with open(json_path, "r", encoding="utf-8") as f:
            data = json.load(f)
            geo = data.get("geoData")
            photo_taken_time = data.get("photoTakenTime", {}).get("timestamp")
            if geo:
                lat = geo.get("latitude")
                lon = geo.get("longitude")
                return (lat, lon), photo_taken_time
    except:
        pass
    return None, None

def find_json_path_for_image(image_path):
    base = os.path.basename(image_path)
    folder = os.path.dirname(image_path)

    suffixes = [
        ".supplemental-metadata.json",
        ".supplemental-meta.json",
        ".supplemental-metada.json",
        ".supplemental-metadat.json",
        ".supplemental-me.json"
    ]

    for suffix in suffixes:
        candidate = image_path + suffix
        if os.path.exists(candidate):
            return candidate

    match = re.match(r'^(.*)\((\d+)\)\.(jpg|jpeg|heic)$', base, re.IGNORECASE)
    if match:
        base_clean = match.group(1).strip()
        suffix_num = match.group(2)
        ext = match.group(3)
        for sfx in suffixes:
            sfx_with_index = sfx.replace(".json", f"({suffix_num}).json")
            alt_json = os.path.join(folder, f"{base_clean}.{ext}{sfx_with_index}")
            if os.path.exists(alt_json):
                return alt_json

    return None

def parse_date_components(date_str):
    try:
        if date_str and len(str(date_str)) == 10 and str(date_str).isdigit():
            dt = datetime.fromtimestamp(int(date_str))
        else:
            dt = datetime.strptime(date_str, "%Y:%m:%d %H:%M:%S")
        return str(dt.year), f"{dt.month:02d}", f"{dt.day:02d}"
    except:
        return "2099", "01", "01"

def spiral_coords(lat, lon, index, step=0.0002):
    angle = index * (math.pi / 3)
    radius = step * (1 + index // 6)
    return lat + radius * math.cos(angle), lon + radius * math.sin(angle)

# === Find all images ===
image_files = []
for dirpath, _, filenames in os.walk(SRC_FOLDER):
    for f in filenames:
        if f.lower().endswith(('.jpg', '.jpeg', '.heic')) and "-edited" not in f.lower():
            image_files.append(os.path.join(dirpath, f))

# === Processing ===
Image.init()
coords_seen = {}
records_ok = []
records_missing = []

for in_path in tqdm(image_files, desc="\U0001f4e6 Обработка изображений", ncols=80):
    source_filename = os.path.basename(in_path)
    ext = os.path.splitext(source_filename)[1].lower()
    album = os.path.basename(os.path.dirname(in_path))
    source_type = None
    lat, lon, date_str = None, None, None

    try:
        if ext in [".jpg", ".jpeg"]:
            gps, date_str = extract_gps_from_exif(in_path)
            if gps:
                lat, lon = gps
                source_type = "exif"

        if lat is None or lon is None:
            json_path = find_json_path_for_image(in_path)
            if json_path:
                gps_json, json_date = extract_gps_from_json(json_path)
                if gps_json is not None:
                    try:
                        lat = float(gps_json[0])
                        lon = float(gps_json[1])
                        source_type = (source_type or "") + "+json"
                        if ext == ".heic":
                            print(f"[HEIC] Найден JSON → lat: {lat}, lon: {lon}, file: {source_filename}")
                    except:
                        lat, lon = None, None
                if not date_str and json_date:
                    date_str = json_date

        year, month, day = parse_date_components(date_str)
        base_name = f"IMG_{year}{month}{day}_{abs(hash(source_filename)) % 10**6}.jpg"
        out_path = os.path.join(OUT_FOLDER, base_name)

        if ext == ".heic":
            print(f"[HEIC] Конвертация: {source_filename} → {base_name}")
            temp_path = "/tmp/temp_output.jpg"
            result = subprocess.run([
                "sips", "-s", "format", "jpeg", in_path, "--out", temp_path
            ], capture_output=True, text=True)

            if result.returncode == 0 and os.path.exists(temp_path):
                image = Image.open(temp_path)
                width, height = image.size
                new_size = (int(width * 0.8), int(height * 0.8))
                image = image.resize(new_size, Image.LANCZOS)
                image.convert("RGB").save(out_path, "JPEG", quality=85)
                os.remove(temp_path)
            else:
                print(f"[HEIC] ❌ Ошибка sips: {result.stderr.strip()}")
                Image.new("RGB", (800, 600), (128, 128, 128)).save(out_path, "JPEG", quality=85)
                source_type = (source_type or "") + "+heic-error"

        else:
            image = Image.open(in_path)
            width, height = image.size
            new_size = (int(width * 0.8), int(height * 0.8))
            image = image.resize(new_size, Image.LANCZOS)
            image.convert("RGB").save(out_path, "JPEG", quality=85)

        row = {
            'filename': base_name,
            'folder': os.path.basename(OUT_FOLDER),
            'latitude': lat,
            'longitude': lon,
            'year': year,
            'month': month,
            'day': day,
            'album': album,
            'source_filename': source_filename,
            'source_type': source_type
        }

        if lat in (None, 0.0) or lon in (None, 0.0):
            row['latitude'] = None
            row['longitude'] = None
            records_missing.append(row)
        else:
            key = (round(lat, 6), round(lon, 6))
            count = coords_seen.get(key, 0)
            if count > 0:
                row['latitude'], row['longitude'] = spiral_coords(lat, lon, count)
            coords_seen[key] = count + 1
            records_ok.append(row)

    except Exception as e:
        print(f"❌ Ошибка при обработке {source_filename}: {e}")
        year, month, day = "2099", "01", "01"
        base_name = f"IMG_{year}{month}{day}_{abs(hash(source_filename)) % 10**6}.jpg"
        records_missing.append({
            'filename': base_name,
            'folder': os.path.basename(OUT_FOLDER),
            'latitude': None,
            'longitude': None,
            'year': year,
            'month': month,
            'day': day,
            'album': album,
            'source_filename': source_filename,
            'source_type': None
        })

# === Save CSVs ===
pd.DataFrame(records_ok).to_csv(CSV_MAIN, index=False)
pd.DataFrame(records_missing).to_csv(CSV_MISSING, index=False)

# === Summary ===
print(f"\n\U0001f4ca Сводка:")
print(f"\U0001f9ee Всего обработано: {len(records_ok) + len(records_missing)}")
print(f"✅ С координатами:   {len(records_ok)}")
print(f"❌ Без координат:    {len(records_missing)}")
print(f"📄 CSV-файлы:        {CSV_MAIN}, {CSV_MISSING}")
print(f"📁 Изображения:      {OUT_FOLDER}")

📦 Обработка изображений:  59%|███████▋     | 1427/2431 [06:35<05:51,  2.86it/s]

[HEIC] Найден JSON → lat: 50.0842092, lon: 14.4240219, file: IMG_1894.HEIC
[HEIC] Конвертация: IMG_1894.HEIC → IMG_20220501_332311.jpg


📦 Обработка изображений:  64%|████████▎    | 1546/2431 [07:11<03:42,  3.98it/s]

[HEIC] Найден JSON → lat: 50.082902600000004, lon: 14.422433299999998, file: IMG_1899.HEIC
[HEIC] Конвертация: IMG_1899.HEIC → IMG_20220501_71799.jpg


📦 Обработка изображений:  70%|█████████    | 1694/2431 [07:56<03:47,  3.24it/s]

...

[HEIC] Найден JSON → lat: 50.075538099999996, lon: 14.437800500000002, file: IMG_1890.HEIC
[HEIC] Конвертация: IMG_1890.HEIC → IMG_20220501_429680.jpg


📦 Обработка изображений:  76%|█████████▊   | 1846/2431 [08:41<02:56,  3.31it/s]

[HEIC] Найден JSON → lat: 50.0861017, lon: 14.416173599999997, file: IMG_1901.HEIC
[HEIC] Конвертация: IMG_1901.HEIC → IMG_20220501_435404.jpg


📦 Обработка изображений: 100%|█████████████| 2431/2431 [10:46<00:00,  3.76it/s]


📊 Сводка:
🧮 Всего обработано: 2431
✅ С координатами:   2424
❌ Без координат:    7
📄 CSV-файлы:        /Users/mloktionov/PycharmProjects/PhotoMaps/csv/photos.csv, /Users/mloktionov/PycharmProjects/PhotoMaps/csv/missing_coords.csv
📁 Изображения:      /Users/mloktionov/PycharmProjects/PhotoMaps/photos
